# FIT5196 Task 2 in Assessment 2
#### Student Name: Boyu Zhang
#### Student ID: 28491300

Date: XX/XX/XXXX

Version: 1.0

Environment: Python 3.6 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas (for dataframe, included in Anaconda Python ) 
* re (for regular expression, included in Anaconda Python 3.6) 
* numpy (for numpy array, included in Anaconda Python 3.6)
* jellyfish (for vairous string similarity matching, obtained by `pip install jellyfish`)
* sklearn (as depency of recordlinkag, obtained by `conda install scikit-learn`)
* recordlinkage (for record matching operations, obatained by `pip install recordlinkage`)

## 1.  Import libraries 

In [62]:
# Code to import libraries as you need in this assessment, e.g.,
import numpy as np
import pandas as pd
import re
import jellyfish
import recordlinkage 

## 2. Load datasets and exploration

In [63]:
df2 = pd.read_csv('./Input/dataset2_integration.csv')
df1 = pd.read_csv('./dataset1_solution.csv')

In [64]:
df1.columns

Index(['Id', 'Title', 'Location', 'ContractType', 'ContractTime', 'Company',
       'Category', 'Salary per annum', 'SourceName', 'OpenDate', 'CloseDate'],
      dtype='object')

In [65]:
df2.columns

Index(['Id', 'Source Name', 'Title', 'location', 'Contract Type',
       'Contract Time', 'Company', 'Category', 'Salary per month', 'OpenDate',
       'CloseDate'],
      dtype='object')

It seems there are two schema level conflicst between the two datasets:
1. There are attributes with same semantics that are not aligned
2. The `Salary` column is inconsistent

## 3. Integrate Datasets



## 3.1 Resolve schema conflicts

First we realign the order of columns in dataset2 and change the unit of `Salary` column

In [66]:
#unify unit of salary and re-align columns by adjusting schema of dataset2
new_salary = df2['Salary per month'].astype(int)*12
df2.drop(columns='Salary per month',inplace=True)
df2.insert(loc=8,column='Salary per annum',value=new_salary)
source = df2['Source Name'] 
df2.drop(columns='Source Name',inplace=True)
df2.insert(loc=8,column='SourceName',value=source)
df2.rename(columns={'location':'Location','Contract Type':'ContractType','Contract Time':'ContractTime'},inplace=True)

Final check before concatenating two datasets:

In [67]:
df1.head()

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,12612628,Engineering Systems Analyst,Dorking,non_specified,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,2012-11-03 00:00:00,2012-12-03 00:00:00
1,12612830,Stress Engineer Glasgow,Glasgow,non_specified,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,2013-01-08 15:00:00,2013-04-08 15:00:00
2,12612844,Modelling And Simulation Analyst,Hampshire,non_specified,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,2013-07-26 15:00:00,2013-09-24 15:00:00
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Surrey,non_specified,permanent,Gregory Martin International,Engineering Jobs,27504,cv-library.co.uk,2012-12-14 00:00:00,2013-03-14 00:00:00
4,12613647,"Pioneer, Miser Engineering Systems Analyst",Surrey,non_specified,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,2013-10-25 00:00:00,2013-12-24 00:00:00


In [68]:
df2.head()

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,69247680,Business Development Exec research / insight ...,London,ft,perm.,BOYCE RECRUITMENT,"PR, Advertising & Marketing Jobs",25500,jobs.guardian.co.uk,2012-03-01 15:00:00,2012-03-31 15:00:00
1,69247682,Audit Senior London,London,ft,contr.,Pro Finance,Finance & Accounting Jobs,43200,icaewjobs.com,2013-03-14 12:00:00,2013-04-13 12:00:00
2,69247685,PR & Social Media Account Executive Top Inter...,Central London,ft,perm.,ECOM RECRUITMENT LTD,"PR, Advertising & Marketing Jobs",23004,jobs.guardian.co.uk,2012-02-14 12:00:00,2012-03-15 12:00:00
3,69247688,Content Manager,London,ft,perm.,NAKAMA LONDON,"PR, Advertising & Marketing Jobs",24996,jobs.guardian.co.uk,2012-10-21 00:00:00,2012-12-20 00:00:00
4,69247694,AV PRODUCTION MANAGER,Worcestershire,ft,perm.,LIVE RECRUITMENT,"PR, Advertising & Marketing Jobs",27504,jobs.guardian.co.uk,2012-02-22 00:00:00,2012-04-22 00:00:00


## 3.2 Resolve data conflicts

### Inconsistence
It seems there are still inconsisent representation of same values in `ContractType` and `ContractTime` columns, as well as inconsistent caplital letters in `Company` column, we fix them now before checking duplication.

Otherwise it may be incorrectly regarded as different strings which will harm the accuracy of duplicates-dropping

In [69]:
df2.ContractType = df2.ContractType.map({'ft':'full_time','pt':'part_time',np.nan:'non_specified'})
df2.ContractTime = df2.ContractTime.map({'perm.':'permanent','contr.':'contract',np.nan:'non_specified'})
df2.Company = df2.Company.str.title()

### Duplication
- We should figure out a global key -- a set of features to identify whether two records are duplicated or not

In [70]:
df1.set_index('Id',inplace=True)
df2.set_index('Id',inplace=True)
df1.index.rename('Id1',inplace=True)
df2.index.rename('Id2',inplace=True)

### Then we should consider comparing similarity of records based on a set of features:
- We should pick out some features that we regard as identical for each record
- From the previous observation we can see that the `Title` of each record is unlikely to be highly similar to others, almost everything from "Year number" to "London" and "20K" can be included in.
- Also the precision of `Salary` is to the last digit, thus they shall be identical to certain extent as well


We need to confirm our inference based on statistics

In [71]:
# test uniqueness level of features in df1
cols = list(df1.columns)
for col in cols:
    print(df1[col].unique().shape[0],col)

25077 Title
477 Location
3 ContractType
3 ContractTime
4705 Company
8 Category
1570 Salary per annum
90 SourceName
2193 OpenDate
2400 CloseDate


In [72]:
# test uniqueness level of features in df2
cols = list(df2.columns)
for col in cols:
    print(df2[col].unique().shape[0],col)

25276 Title
475 Location
3 ContractType
3 ContractTime
5546 Company
8 Category
1505 Salary per annum
92 SourceName
2193 OpenDate
2395 CloseDate


#### Decision
So we pick the combination of features with the top level of uniqueness in both datasets:
`Title`,`Company`,`Salary per annum`,`OpenDate`,`CloseDate`

**One more tip**:

It is unsual that "np.nan == np.nan" will return **False**, we recall from task1 that there are quite a number of null values in df1 which will harm the performance of our deduplication process.

Thus we replace the null value with same string "Unknown"

In [73]:
df1.Company.fillna('Unknown',inplace=True)
df2.Company.fillna('Unknown',inplace=True)

### Create record pairs

First we need to check whether there are potential duplicated records in each dataset itself.

In [74]:
indexer = recordlinkage.BlockIndex(on=['Title','Company','Salary per annum','OpenDate','CloseDate'])
pairs = indexer.index(df1)
len(pairs)

ValueError: Duplicated level name: "Id1", assigned to level 1, is already used for level 0.

In [75]:
indexer = recordlinkage.BlockIndex(on=['Title','Company','Salary per annum','OpenDate','CloseDate'])
pairs = indexer.index(df2)
len(pairs)

ValueError: Duplicated level name: "Id2", assigned to level 1, is already used for level 0.

This means no potential duplicated records are found respectively in each dataset.

Then we need to find potential pairs in both datasets

In [76]:
indexer = recordlinkage.BlockIndex(on=['Title','Company','Salary per annum','OpenDate','CloseDate'])
pairs = indexer.index(df1,df2)
len(pairs)

86

Then we need to compute the simliarity of each record pairs

In [77]:
compare_cl = recordlinkage.Compare()

In [78]:
# get mean value of salary to provide reference to salary similarity comparison
df1['Salary per annum'].mean()

34364.17474179527

We set a few threshold on giving the similarity score, if similarity level exceeds the threshold, we give it 1, otherwise 0,

In [79]:
# Compute the similarity score of each selected feature between record pairs
compare_cl = recordlinkage.Compare()
# since the title is too customized we try not be too strict on them
compare_cl.string('Title','Title',threshold=0.6,method='levenshtein',label='Title')
#Company is not that unique, so we set higher threshold
compare_cl.string('Company','Company',threshold=0.8,label='Company')
# The salary comparison is based on gaussian distribution, the scale is based on the mean value
compare_cl.numeric('Salary per annum','Salary per annum',scale=1000,method='gauss',label='Salary')
compare_cl.string('OpenDate','OpenDate',threshold=0.95,label='OpenDate')
compare_cl.string('CloseDate','CloseDate',threshold=0.95,label='CloseDate')
features = compare_cl.compute(pairs, df1, df2)

In [80]:
#for all 5 features we regard two record with score higher than 4 as duplicated
matches = features[features.sum(axis=1) > 4]
matches

,,Title,Company,Salary,OpenDate,CloseDate
Id1,Id2,,,,,
44363005,72446581,1.0,1.0,1.0,1.0,1.0
46632961,70176593,1.0,1.0,1.0,1.0,1.0
46637204,72032810,1.0,1.0,1.0,1.0,1.0
50881828,71680650,1.0,1.0,1.0,1.0,1.0
60685289,71850757,1.0,1.0,1.0,1.0,1.0
62282446,71774964,1.0,1.0,1.0,1.0,1.0
64146768,71749988,1.0,1.0,1.0,1.0,1.0
64353737,69896092,1.0,1.0,1.0,1.0,1.0
65171288,72625397,1.0,1.0,1.0,1.0,1.0


Let's have a quick check does the pair looks really close to each other?

In [81]:
df1.loc[68838100]

Title               Clinical Project Manager (Late Phase)  Germany
Location                                                        UK
ContractType                                             full_time
ContractTime                                             permanent
Company                                           Barrington James
Category                                 Healthcare & Nursing Jobs
Salary per annum                                             60000
SourceName                                       strike-jobs.co.uk
OpenDate                                       2012-01-02 12:00:00
CloseDate                                      2012-02-01 12:00:00
Name: 68838100, dtype: object

In [82]:
df2.loc[71625551]

Title               Clinical Project Manager (Late Phase)  Germany
Location                                                        UK
ContractType                                             full_time
ContractTime                                             permanent
Company                                           Barrington James
Category                                 Healthcare & Nursing Jobs
Salary per annum                                             60000
SourceName                                       strike-jobs.co.uk
OpenDate                                       2012-01-02 12:00:00
CloseDate                                      2012-02-01 12:00:00
Name: 71625551, dtype: object

Seems the result is not bad, so we can now regard these 73 records as duplicated so we can drop them in either dataset.

In [83]:
#get the list of ids for records to delete in df1(either way is valid)
id_to_delete = list(features.index.get_level_values('Id1'))

In [84]:
#drop the duplicated rows in dataset1
df1 = df1.drop(index=id_to_delete)

Last check of no duplicated records detected

In [85]:
#last check of potential duplicated record pairs
pairs = indexer.index(df1,df2)
len(pairs)

0

In [86]:
#undo changes to make dataset looks the same as original
df1.reset_index(level=0, inplace=True)
df2.reset_index(level=0, inplace=True)

In [87]:
# combine 2 datasets
df = pd.concat((df1,df2))

/home/zby0902/anaconda3/envs/5196/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  from ipykernel import kernelapp as app


In [88]:
# third check of no duplication existing after the merge of datasets
df.duplicated(['Title','Company','Salary per annum','OpenDate','CloseDate']).sum()

0

Since there are no more duplicates, we can output the file

In [90]:
df

,Category,CloseDate,Company,ContractTime,ContractType,Id1,Id2,Location,OpenDate,Salary per annum,SourceName,Title
0,Engineering Jobs,2012-12-03 00:00:00,Gregory Martin International,permanent,non_specified,12612628.0,NaN,Dorking,2012-11-03 00:00:00,24996,cv-library.co.uk,Engineering Systems Analyst
1,Engineering Jobs,2013-04-08 15:00:00,Gregory Martin International,permanent,non_specified,12612830.0,NaN,Glasgow,2013-01-08 15:00:00,30000,cv-library.co.uk,Stress Engineer Glasgow
2,Engineering Jobs,2013-09-24 15:00:00,Gregory Martin International,permanent,non_specified,12612844.0,NaN,Hampshire,2013-07-26 15:00:00,30000,cv-library.co.uk,Modelling And Simulation Analyst
3,Engineering Jobs,2013-03-14 00:00:00,Gregory Martin International,permanent,non_specified,12613049.0,NaN,Surrey,2012-12-14 00:00:00,27504,cv-library.co.uk,Engineering Systems Analyst / Mathematical Mod...
4,Engineering Jobs,2013-12-24 00:00:00,Gregory Martin International,permanent,non_specified,12613647.0,NaN,Surrey,2013-10-25 00:00:00,24996,cv-library.co.uk,"Pioneer, Miser Engineering Systems Analyst"
5,Accounting & Finance Jobs,2012-09-22 00:00:00,Brite Recruitment,permanent,non_specified,19047429.0,NaN,East Midlands,2012-07-24 00:00:00,21000,cv-library.co.uk,Trainee Mortgage Advisor East Midlands
6,Healthcare & Nursing Jobs,2013-04-27 12:00:00,Matchbox Recruiting Ltd,permanent,non_specified,20199757.0,NaN,Witney,2013-03-28 12:00:00,37500,cv-library.co.uk,"Project Engineer, Pharmaceutical"
7,Hospitality & Catering Jobs,2013-12-26 00:00:00,Chef Results,non_specified,non_specified,20797143.0,NaN,Derby,2013-10-27 00:00:00,15996,caterer.com,Chef De Partie Award Winning Restaurant Exce...
8,Engineering Jobs,2013-09-14 00:00:00,Asset Appointments,permanent,non_specified,22579462.0,NaN,Gateshead,2013-08-15 00:00:00,21996,cv-library.co.uk,Quality Engineer
9,Hospitality & Catering Jobs,2012-08-28 00:00:00,Chef Results,non_specified,non_specified,22933091.0,NaN,UK,2012-08-14 00:00:00,18000,caterer.com,Chef De Partie Award Winning Dining Live In ...


In [89]:
#output
df.to_csv('./dataset1_dataset2_solution.csv')

## 4. Summary
There are various way(algorithms) of measure similarity of strings and numeric values, we need to make a lot of choices. Thus the result can be quite dependent on the measures and features we pick.
### Reference
 1. recordlinkage. Retrieved from http://recordlinkage.readthedocs.io/en/latest/notebooks/data_deduplication.html
 2. How to get index values pandas.Retrieved from: https://stackoverflow.com/questions/18358938/get-row-index-values-of-pandas-dataframe-as-list
 3. How to convert pandas index in a dataframe to a column? Retrieved from:https://stackoverflow.com/questions/20461165/how-to-convert-pandas-index-in-a-dataframe-to-a-column